import package

In [5]:
import os
import itertools

import numpy as np
import pandas as pd
import pyproj
import geopandas as gpd
import shapely
import fiona

from tqdm import tqdm
import networkx as nx

from sqr.core.shape import make_gdf_square_data, find_neighbor_shapes
from sqr.core.shape import label2coord, polygon_from_north_east
from sqr.pre_assign import pre_partition_area, assign_cells_partition, merge_insufficient

%matplotlib inline 

# Preprocess data
Load kvadratnet with population data into HDF store: 'data/parsed/kvadrat_data.hdf'

In [ ]:
pop_file = 'DST/Antal personer pr. celle.xlsx'

for dst_sheetname in ['_10km','_1km','_100m']:
    print(dst_sheetname)
    in_df = pd.read_excel(pop_file, sheetname=dst_sheetname)
    in_df.to_hdf('DST/personer_celler.hdf', key = dst_sheetname)

Make HDF format of 100x100 kvadratnet and append data

Note this should be only run the first time using this notebook

In [3]:
dk = gpd.read_file('raw/DKN_100m_euref89.shp').iloc[:,:1]
dk_bornholm = gpd.read_file('data/raw/DKN_bholm_100m_euref89.shp').iloc[:,:1]

pd.concat([dk, dk_bornholm],
          ignore_index=True)\
  .KN100mDK\
  .to_hdf(data='data/parsed/kvadrat_data.hdf', 
          key='cells100')

KN100mDK = pd.read_hdf('data/parsed/kvadrat_data.hdf', key='cells100')

coords = pd.DataFrame(np.array(KN100mDK.str.split('_').tolist())[:,1:], 
                      columns = ['n','e'])


coords['e_cent'] = (coords.e.astype(int)*100+50).astype(np.int32)    
coords['n_cent'] = (coords.n.astype(int)*100+50).astype(np.int32)

p1 = pyproj.Proj(fiona.crs.from_epsg(25832))
p2 = pyproj.Proj(fiona.crs.from_epsg(4326))    
    
gps_coords = pyproj.transform(p1, p2, coords.e_cent.values, coords.n_cent.values)
gps_coords = pd.DataFrame(np.array(gps_coords).T, columns = ['lon_cent','lat_cent'])

all_coords = pd.concat([KN100mDK,coords,gps_coords],axis=1)

all_coords.to_hdf('data/parsed/kvadrat_data.hdf', key='cells100_data')

# Assign square to municipality shapes

### Load data
Load square cell population data and perform various calculations.

In [2]:
pers = pd.read_hdf('data/parsed/personer_celler.hdf', key='_100m').iloc[1:]
year_cols = list(map(str,range(1986,2016)))

pers_years = pers[year_cols]
pers['minimum'] = pers_years.min(axis=1)
pers['mean'] = pers_years.fillna(0).mean(axis=1)

# new construction year
pers['zero'] = pers_years.isnull().max(axis=1)
zeros = pers_years[pers.zero].isnull()
zeros_t = zeros.T
inhabit = (zeros_t.shift(1).fillna(False) & (~zeros_t))
inhabit_single = (inhabit.sum(axis=0)==1) & (~zeros.iloc[:,-1])
inhabit_year = zeros_t\
                .loc[:,inhabit_single]\
                .idxmin()\
                .rename('inhabit_year')\
                .astype(int)
pers = pers.join(inhabit_year)


Load square municipal data and combine with square cell population

In [ ]:
kommuner = gpd.read_file('data/shape/KOMMUNE.shp')

pers = pd.read_hdf('data/parsed/personer_celler.hdf', key='_100m').iloc[1:]
year_cols = list(map(str,range(1986,2016)))

pers_years = pers[year_cols]
pers['minimum'] = pers_years.min(axis=1)
pers['mean'] = pers_years.fillna(0).mean(axis=1)

# new construction year
pers['zero'] = pers_years.isnull().max(axis=1)
zeros = pers_years[pers.zero].isnull()
zeros_t = zeros.T
inhabit = (zeros_t.shift(1).fillna(False) & (~zeros_t))
inhabit_single = (inhabit.sum(axis=0)==1) & (~zeros.iloc[:,-1])
inhabit_year = zeros_t\
                .loc[:,inhabit_single]\
                .idxmin()\
                .rename('inhabit_year')\
                .astype(int)
pers = pers.join(inhabit_year)


        
        
all_squares = pd.read_hdf('data/parsed/kvadrat_data.hdf', key='cells100_data')
all_squares.rename(columns = {'lon_cent':'lon','lat_cent':'lat'}, inplace=True)
all_squares.e = all_squares.e.astype(int)
all_squares.n = all_squares.n.astype(int)

all_gdf = make_gdf_square_data(all_squares)

Assign square cells to municipalities (municipalities after kommunalreformen 2007)

In [ ]:
assignments = assign_cells_partition(kommuner, all_gdf)

assignment_dict = assignments\
                    .groupby('assignment')\
                    .apply(lambda g: g.index.tolist())\
                    .to_dict()

extra_cols = ['minimum','mean','shift_year','ddkncelle100m']            
            
for idx in assignment_dict.keys():
    assignment_idxs = assignment_dict[idx]
    
    out_df = all_gdf\
            .loc[assignment_idxs]\
            .drop('geometry', axis=1)\
            .reset_index()\
            .rename(columns={'index':'square_idx'})
    
    out_df = out_df.merge(pers[extra_cols+year_cols], 
                          right_on='ddkncelle100m',
                          left_on='KN100mDK', 
                          how='left')\
                .drop('ddkncelle100m',axis=1)
    
    pd.DataFrame(out_df).to_hdf('data/parsed/sqr_mun.hdf', key='sqidx%i'% idx)    
    
# check no cells is overlapping for ANY pair of municipality indices

errors = []

for (i1,i2) in mun_neighbor:
    cells1 = cell_assignment[i1]['within'] + list(cell_assignment[i1]['touching'])
    cells2 = cell_assignment[i2]['within'] + list(cell_assignment[i2]['touching'])
    
    if np.intersect1d(cells1,cells2).size>0:
        errors+= [(i1,i2)]
        
print (errors)        

# Partition municipality shapes into chunks

In [ ]:
kommuner = gpd.read_file('data/shape/KOMMUNE.shp')

mun_pop_min = {}
mun_pop_avg = {}
mun_cell_count = {}

for idx in kommuner.index:
    mun_data = \
        pd.read_hdf('data/parsed/sqr_mun.hdf', key='sqidx%i'% idx)
    mun_pop_min[idx] = mun_data.minimum.sum()    
    mun_pop_avg[idx] = mun_data['mean'].sum()    
    mun_cell_count[idx] = mun_data.shape[0]
                
kommuner['minimum_total'] = pd.Series(mun_pop_min)
kommuner['mean_total'] = pd.Series(mun_pop_avg)
kommuner['cell_count'] = pd.Series(mun_cell_count)

kommuner['to_assign'] = kommuner.minimum_total>100

In [ ]:
select =  kommuner[(kommuner.to_assign) & (kommuner.cell_count>25000)]

for idx in tqdm(select.index.tolist()):
    print(idx)
    origin_geom= kommuner.iloc[idx].geometry
    mun_df = pd.read_hdf('data/parsed/sqr_mun.hdf', 'sqidx%i' % idx)
    mun_gdf = make_gdf_square_data(mun_df)
    pre_part = pre_partition_area(mun_df, origin_geom)
    
    pre_part_suff = merge_insufficient(pre_part)
    
    assignment = assign_cells_partition(pre_part, mun_gdf)

    mun_df = mun_df\
                .join(assignment)\
                .drop('geometry', axis=1)

    for sub_idx, sub_df in mun_df.groupby('assignment'):
        out_key = 'sqidx%i_%i' % (idx, sub_idx)
        sub_df.to_hdf('data/parsed/sqr_mun_sub.hdf', key = out_key)